# RF including Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from helper import * 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
# import helper

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

# Import data and clean with wrapper

In [4]:
# importing data
housing = pd.read_csv('Ames_Housing_Price_Data.csv', index_col=0,low_memory = False)
# data processing
train, test = data_processing_wrapper(housing,
                                               num_to_cat_list = ['MSSubClass','MoSold'],
                                             remove_PID = False
                                        )

# feature engineering wrapper
train, test = feature_engineering_wrapper(train, test)

# importing school feature
schools = pd.read_csv('schoolFeatures.csv',index_col = 0)
school_keep = [
    'PID',
    'closestSchool'
]
schools = schools[school_keep]

# merge school feature onto original data set.
train = train.merge(schools, how = 'left', left_on = 'PID', right_on = 'PID')
test = test.merge(schools, how = 'left', left_on = 'PID', right_on = 'PID')

train = train.dropna(subset=['closestSchool'])
train = train.reset_index(drop=True)

test = test.dropna(subset=['closestSchool'])
test = test.reset_index(drop=True)

In [5]:
#housing started out with 81 columns (2580 rows)
#feature engineering - ended up with 129 columns (including PID and SalePrice)-- (added 48 new columns) (2477 rows between train and test)

# Split into predictors and target

In [6]:
train_raw = train.copy()
test_raw = test.copy()

train_X = train_raw.drop(['SalePrice','PID'],axis='columns')
train_y = np.log(train_raw['SalePrice'])
test_X = test_raw.drop(['SalePrice','PID'],axis='columns')
test_y = np.log(test_raw['SalePrice'])

In [7]:
train_y_log = train_y.rename('LogSalePrice')
test_y_log = test_y.rename('LogSalePrice')

In [20]:
cat_feats = train_X.select_dtypes(['object']).columns.to_list()
num_feats = train_X.select_dtypes(['int64','float64']).columns.to_list()

In [21]:
print(len(cat_feats))
print(len(num_feats))
len(train_X.columns.to_list())

32
95


127

In [19]:
set(train_X.dtypes)

{dtype('int64'), dtype('float64'), dtype('O')}

In [17]:
odd_cols = [col for col in train_X.columns.to_list() if ((col not in cat_feats) and (col not in num_feats))]
train_X[odd_cols].dtypes

GrLivArea                   int64
LotArea                     int64
Alley                       int64
OverallQual                 int64
OverallCond                 int64
YearBuilt                   int64
YearRemodAdd                int64
ExterQual                   int64
ExterCond                   int64
BsmtQual                    int64
BsmtCond                    int64
BsmtExposure                int64
HeatingQC                   int64
1stFlrSF                    int64
2ndFlrSF                    int64
LowQualFinSF                int64
FullBath                    int64
HalfBath                    int64
BedroomAbvGr                int64
KitchenAbvGr                int64
KitchenQual                 int64
TotRmsAbvGrd                int64
Fireplaces                  int64
FireplaceQu                 int64
GarageFinish                int64
GarageQual                  int64
GarageCond                  int64
PavedDrive                  int64
WoodDeckSF                  int64
OpenPorchSF   

In [9]:
len(cat_feats) # 32 categorical features (43 originally in housing dataset)
len(num_feats) #95 numeric features (38 originally)

35

# Dummify 

In [ ]:
#Preprocessing / Dummification
preprocessor = ColumnTransformer(transformers=[
    ('tf1',OneHotEncoder(sparse=False, handle_unknown='ignore'), cat_feats)],remainder='passthrough')

train_X_transformed = preprocessor.fit_transform(train_X)

In [ ]:
#Get one-hot encoded column names 
columns_transformed = preprocessor.named_transformers_['tf1'].get_feature_names(input_features= cat_feats)
new_columns = list(columns_transformed)+num_feats

#Place one-hot encoded train X into dataframe 
train_X_transformed = pd.DataFrame(train_X_transformed,columns=new_columns)

#Repeat for test X 
test_X_transformed = preprocessor.transform(test_X)
test_X_transformed = pd.DataFrame(test_X_transformed,columns=new_columns)

# Filter down Features Based off of LASSO 

In [ ]:
#Features selected by LASSO model (non-zero coefficients)
coef_df = pd.read_csv('lasso_coef.csv',index_col=0) #Hayden shared this on Slack

In [ ]:
selected_features = list(coef_df['features'])

In [ ]:
print('Number of LASSO selected features: '+ str(len(selected_features)))

In [ ]:
print('Number of total features after preprocessing: ' + str(len(train_X_transformed.columns.to_list())))

In [ ]:
#Filter dataframes down to the select_features 
train_X= train_X_transformed[selected_features]
test_X = test_X_transformed[selected_features]

# Random Forest Modeling 

In [ ]:
#Modeling 

In [ ]:
#1. No Tuning

In [ ]:
rf = RandomForestRegressor(random_state=0)

In [ ]:
rf.fit(train_X, train_y_log)

In [ ]:
print('Cross Val score: ', cross_val_score(rf, train_X, train_y_log, cv=3))
print('Cross Val score mean: ', cross_val_score(rf, train_X, train_y_log, cv=3).mean())
print('Train score: ',rf.score(train_X,train_y_log))
print('Test score: ',rf.score(test_X,test_y_log))

In [ ]:
#2. With Tuning 

In [ ]:
# Number of trees in random forest
n_estimators = [100,200,400,600,1000]
# Number of features to consider at every split
max_features = list(range(10,140,20)) + ['auto','sqrt']
# Maximum number of levels in tree
max_depth = list(range(10,111,10)) + ['None']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
random_grid

In [ ]:
rf = RandomForestRegressor()